In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/Users/keemsunguk/Projects/EssayGrader/')

In [3]:
import spacy
from spacy import displacy

In [4]:
import json

In [5]:
import re

In [6]:
from egrader.preprocess import Preprocess

In [42]:
from egrader.db_util import DBUtil
db_util = DBUtil(local_db=True)
db_util.describe_db()

==== LOCAL ====
['essay']
Total Essay: 30498
Total SAT: 3310
Total TOEFL: 4614
Total GRE: 6688
==== REMOTE ====
['essays']
Total Essay: 29889
Total SAT: 3280
Total TOEFL: 4613
Total GRE: 6663


In [40]:
short_essay = [e['recno'] for e in db_util.remote_ec.find({}) if len(e['essay']) < 2]

In [ ]:
db_util.remote_ec.delete_many({'recno':{'$in':short_essay}})

In [15]:
db_util.local_ec.delete_many({'essay': {'$eq':None}})

In [8]:
db_util.remote_ec.find_one({}).keys()

dict_keys(['_id', 'essay', 'time_stamp', 'user_id', 'wcount', 'topic', 'edited', 'type', 'created', 'recno', 'editor', 'rate2', 'rate', 'reviewed'])

In [23]:
txt = db_util.remote_ec.find_one({})['essay']
tmp = re.finditer('\\r\\n\\r\\n', txt)

In [27]:
pp = Preprocess("")
pp.clean_html()

Loaded model 'en_core_web_lg'


''

In [28]:
toefl_df = db_util.get_spacy_labeled_essays('TOEFL', merge_0_1=False, with_topic=True)

In [29]:
for r, v in toefl_df.iterrows():
    toefl_df.iloc[r, 0] = pp.clean_html(v[0])
    print(toefl_df.iloc[r, 0])

A company has announced that it wishes to build a large factory near your community. Discuss the advantages and disadvantages of this new influence on your community. Do you support or oppose the factory? Explain your position.
Which is more important for success: the natural ability you are born with or hard work? Explain your opinion, using specific reasons and examples.
Which is more important for success: the natural ability you are born with or hard work? Explain your opinion, using specific reasons and examples.
Movies are popular all over the world. Explain why movies are so popular. Use reasons and specific examples to support your answer.
Movies are popular all over the world. Explain why movies are so popular. Use reasons and specific examples to support your answer.
Some people say that the Internet provides people with a lot of valuable information. Others think access to so much information creates problems. Which view do you agree with? Use specific reasons and examples t

In [11]:
topic_doc = pp.make_doc_obj(toefl_df.iloc[1, 0])

In [12]:
displacy.render(topic_doc, style="ent", jupyter=True)

/Users/keemsunguk/opt/anaconda3/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


In [14]:
noun_chunks = list(topic_doc.noun_chunks)
topic_doc.vector[1]

0.2246625

In [15]:
essay_doc = pp.make_doc_obj(toefl_df.iloc[1, 1])
displacy.render(essay_doc, style="ent", jupyter=True)

In [16]:
essay_doc.similarity(topic_doc)

0.9570417662732015

In [17]:
topic_doc.to_json()

{'text': 'Which is more important for success: the natural ability you are born with or hard work? Explain your opinion, using specific reasons and examples.',
 'ents': [],
 'sents': [{'start': 0, 'end': 88}, {'start': 89, 'end': 147}],
 'tokens': [{'id': 0,
   'start': 0,
   'end': 5,
   'pos': 'DET',
   'tag': 'WDT',
   'dep': 'nsubj',
   'head': 1},
  {'id': 1,
   'start': 6,
   'end': 8,
   'pos': 'AUX',
   'tag': 'VBZ',
   'dep': 'ROOT',
   'head': 1},
  {'id': 2,
   'start': 9,
   'end': 13,
   'pos': 'ADV',
   'tag': 'RBR',
   'dep': 'advmod',
   'head': 3},
  {'id': 3,
   'start': 14,
   'end': 23,
   'pos': 'ADJ',
   'tag': 'JJ',
   'dep': 'acomp',
   'head': 1},
  {'id': 4,
   'start': 24,
   'end': 27,
   'pos': 'ADP',
   'tag': 'IN',
   'dep': 'prep',
   'head': 3},
  {'id': 5,
   'start': 28,
   'end': 35,
   'pos': 'NOUN',
   'tag': 'NN',
   'dep': 'pobj',
   'head': 4},
  {'id': 6,
   'start': 35,
   'end': 36,
   'pos': 'PUNCT',
   'tag': ':',
   'dep': 'punct',
   'hea

In [18]:
train_data = [(v[0], v[1]) for k, v in toefl_df.iterrows()]

In [19]:
from collections import Counter

In [20]:
Counter(list(toefl_df[2].values))

Counter({'4': 2729, '3': 737, '5': 470, '1': 92, '2': 523, '6': 133})

In [24]:
a = json.dumps(db_util.remote_ec.find_one({}, {'_id':0}))
a

'{"essay": "The letter from a committee contains an assertion that there should be restrictions on landscaping and housepainting to raise the property values. It show an example of Brookville and assumes that the increase in the property values in Brookville is due to the set of restrictions on landscaping and housepainting which was adopted seven years ago. But there is no evidence that those regulations actually contributed to raising the property values. The property values are affected by many factors and seven years is a long time for other factors rather than landscaping and housepainting could have affected the property values. Before adopting the own set of restrictions, the committee should study the direct relationship between the restrictions in Brookville and the increase in the property values.\\r\\n\\r\\n     The property values can be affected by many factors. If they were increased, it could be due to a new factory built in a nearby village or a new department store in 